# Project-15
## Nội dung :
Có N lớp 1,2,..., N cần được xếp thời khóa biểu. Mỗi lớp i có t(i) là số tiết và g(i) là giáo viên đã được phân công dạy lớp đó và s(i) là số sinh viên của lớp 
Có M phòng học 1, 2, ..., M, trong đó c(i) là số chỗ ngồi của phòng i  Trong tuần có 5 ngày (từ thứ 2 đến thứ 5), mỗi ngày chia thành 12 tiết (6 tiết sáng và 6 tiết chiều).  
Hãy lập thời khóa biểu (xác định ngày, tiết và phòng gán cho mỗi lớp)  
Hai lớp có chung giáo viên thì phải xếp thời khóa biểu tách rời nhau  
Số sinh viên trong mỗi lớp phải nhỏ hơn hoặc bằng số chỗ ngồi của phòng học

## Dữ liệu đầu vào :
File data: "data.txt"
> Dòng 1: ghi N và M <br>
> Dòng i+1 (i = 1,…, N): ghi t(i), g(i) và s(i)  
> Dòng N+2: ghi c(1), c(2), …, c(M)

## Đặt biến
n Lớp: l từ 0,1,2...n-1
- T[l] Số tiết - tuple
-	G[l] Giáo viên dạy lớp đó - tuple
-	S[l] số học sinh - tuple <br>

Từ G tạo ra 1 list D_G có index là giáo viên-1, element là list các lớp giáo viên đó dạy.


m Phòng: p từ 0,1,2,..m-1
-	C - số chỗ trong tất cả các phòng
-	T’ - Số tiết trong 10 buổi của tất cả các phòng


Phòng số p có:
- C[p] Số chỗ trong phòng
- T[p][b] Số tiết còn lại trong buổi thứ b
VD:
2 phòng 1,2
3 buổi 1,2,3
C = [40,30]
T = [[6,4,5],[3,2,5]]
=> Phòng số 1 có 40 chỗ và còn trống 4 tiết vào buổi thứ 2


### Input 

In [54]:
from ortools.sat.python import cp_model
from copy import deepcopy
from random import randint

def input(FileName):
    f = open(FileName,'r')
    global so_lop, so_phong, so_buoi, so_tiet, T,S,G,D_G,C
    so_buoi, so_tiet= 10, 6
    so_lop, so_phong =  (int(i) for i in f.readline().split())
    T,S,G,D_G,C = [],[],[],[],[]
    for c in range(so_lop):
        tiet, gv, so_hs = (int(i) for i in f.readline().split())
        T.append(tiet)
        S.append(so_hs)
        G.append(gv)
        if len(D_G) < gv:
            D_G.append([c])
        else:
            D_G[gv-1].append(c)
    C = [int(i) for i in f.readline().split()]


FileName= 'data.txt'
input(FileName)
all_gv = range(len(D_G))
all_p = range(so_phong)
all_b = range(so_buoi)
all_t = range(so_tiet)
all_l = range(so_lop)

[4, 4, 3, 4, 4, 2, 3, 4, 3, 3, 3, 3, 2, 3, 3]


## Constraint 1
Nếu lớp được xếp vào phòng, số học sinh phải nhỏ hơn số chỗ ngồi <br>
>$\forall l \in \text{ all_l}, \forall p \in \text{ all_p}, \forall b \in \text{all_b}, \forall t \in \text{all_t}:$ <br>
>$\displaystyle \text{If } S[l] > C[p]: lc(l,p,b,t) = 0 $


## Constraint 2
Trong cùng 1 thời gian:
 - 1 phòng chỉ chứa nhiều nhất 1 lớp <br>
   >$\displaystyle \sum_{l \in \text{ all_l}} lc(l,p,b,t) \le 1, \forall p \in \text{ all_p}, \forall b \in \text{all_b}, \forall t \in \text{all_t}$

- 1 giáo viên không được dạy 2 lớp khác nhau <br>
>   $\displaystyle \sum_{l \in \text{ D_G[g]}} \text{ }\sum_{p \in \text{ all_p}} lc(l,p,b,t) \le 1, \forall g \in \text{all_gv},\forall b \in \text{all_b}, \forall t \in \text{all_t}$


## Constraint 3
Mỗi lớp:
- Học đủ số tiết<br>
>$\displaystyle \sum_{b \in \text{ all_b}} \text{ }\sum_{p \in \text{ all_p}} \text{ }\sum_{t \in \text{ all_t}} lc(l,p,b,t) = T[l], \forall l \in \text{all_l}$
-  Chỉ học trong 1 phòng, 1 buổi duy nhất và các tiết phải liền nhau trong buổi đó<br>
>$\forall l \in \text{ all_l}, \forall p \in \text{ all_p}, \forall b \in \text{all_b}:$ <br>
>$\displaystyle \text{If} \sum_{t \in \text{ all_t}} lc(l,p,b,t) \neq 0: \exists i \in [0 \text{ ; } 6-T[l]]: \sum_{k=i} ^{i+T[l]-1} lc(l,p,b,k) = T[l] $

>$\displaystyle \prod_{b \in \text{ all_b}} \text{ } \prod_{p \in \text{ all_p}} (T[l]-\sum_{t \in \text{ all_t}} lc(l,p,b,t)) = 0 , \forall l \in \text{all_l}$

### Biến lựa chọn - dict
(Lớp,phòng,buổi,tiết) = 0 hoặc 1 - (6 * 10 * n * m) biến


Lớp 5 xếp vào phòng 1 trong buổi 2 tiết 1, Lớp 6 không xếp vào phòng 1 trong buổi 2 tiết 2 - { (5,1,2,1):1, (6,1,2,2):0 }

In [55]:
def generate_decision_var(algo):
    ''' Generate descision variable base on specify algorithm
    algo: string - 'o': or-tools, 'b': backtracking, 'h': heuristic'''
    global lc
    lc = {}
    for l in all_l:
        for p in all_p:
            for b in all_b:
                for t in all_t:
                    if algo == 'o':
                        lc[(l, p, b, t)] = model.NewBoolVar('lc_l%ip%ib%it%i' %(l, p, b, t))
                    else:
                        lc[(l,p,b,t)]=0
    if algo == 'h':
        global Count, starting
        Count = [0 for i in all_l] # dùng để chỉ những lớp chưa xếp và đã xếp được bao nhiêu tiết
        starting = [] # kết quả (lớp học lúc nào ở đâu)
    if algo == 'b':
        global remain_periods
        remain_periods = [[so_tiet for b in all_b] for p in all_p]

In [56]:
def Backtracking(lc, rmp, rmc):
    '''at first: rmc (remaining_classses) = so_lop
                 rmp = remain_periods '''
    if satisfied_c2(lc, range(rmc,so_lop)): #Constraint 2
        if not rmc:
            return lc
        l = rmc - 1
        for p in all_p:
            for b in all_b:
                if T[l] <= rmp[p][b] and S[l] <= C[p]: #Constraint 3 and 1
                    lc_copy, rmp_copy = deepcopy(lc), deepcopy(rmp)
                    start = so_tiet - rmp[p][b]
                    for t in range(start, start + T[l]):
                        lc_copy[(l,p,b,t)] = 1
                    rmp_copy[p][b] -= T[l]
                    next_state = Backtracking(lc_copy, rmp_copy, rmc - 1)
                    if next_state: return next_state
    return False

def satisfied_c2(lc,all_l):
    '''all_l: Classes which are arranged'''
    for b in all_b:
        for t in all_t:
            for g in all_gv:
                if sum(sum(lc[(l,p,b,t)] for p in all_p) for l in D_G[g]) > 1: #2b
                    return False
            for p in all_p:
                if sum(lc[(l,p,b,t)] for l in all_l) > 1: #2a
                    return False
    return True


In [57]:
def test_Backtracking():
    generate_decision_var('b')
    result = Backtracking(lc, remain_periods, so_lop)
    if not result:
        print("Can't")
    for b in all_b:
        print('Buoi',b)
        for p in all_p:
            print('-Phong',p)
            for l in all_l:
                for t in all_t:
                    if result[(l,p,b,t)] == 1:
                        print('--Lop',l,'hoc giao vien',G[l],'tiet',t)
test_Backtracking()

Buoi 0
-Phong 0
--Lop 4 hoc giao vien 3 tiet 0
--Lop 4 hoc giao vien 3 tiet 1
--Lop 4 hoc giao vien 3 tiet 2
--Lop 4 hoc giao vien 3 tiet 3
-Phong 1
--Lop 13 hoc giao vien 1 tiet 3
--Lop 13 hoc giao vien 1 tiet 4
--Lop 13 hoc giao vien 1 tiet 5
--Lop 14 hoc giao vien 1 tiet 0
--Lop 14 hoc giao vien 1 tiet 1
--Lop 14 hoc giao vien 1 tiet 2
Buoi 1
-Phong 0
--Lop 3 hoc giao vien 3 tiet 0
--Lop 3 hoc giao vien 3 tiet 1
--Lop 3 hoc giao vien 3 tiet 2
--Lop 3 hoc giao vien 3 tiet 3
-Phong 1
--Lop 11 hoc giao vien 1 tiet 2
--Lop 11 hoc giao vien 1 tiet 3
--Lop 11 hoc giao vien 1 tiet 4
--Lop 12 hoc giao vien 2 tiet 0
--Lop 12 hoc giao vien 2 tiet 1
Buoi 2
-Phong 0
--Lop 1 hoc giao vien 3 tiet 0
--Lop 1 hoc giao vien 3 tiet 1
--Lop 1 hoc giao vien 3 tiet 2
--Lop 1 hoc giao vien 3 tiet 3
-Phong 1
--Lop 9 hoc giao vien 1 tiet 3
--Lop 9 hoc giao vien 1 tiet 4
--Lop 9 hoc giao vien 1 tiet 5
--Lop 10 hoc giao vien 3 tiet 0
--Lop 10 hoc giao vien 3 tiet 1
--Lop 10 hoc giao vien 3 tiet 2
Buoi 3
-Phon